<font size="+3"><strong>4 Earthquake Damage in Nepal</strong></font>

<font size="+2"><strong>4.5 Earthquake Damage in Kavrepalanchok 🇳🇵</strong></font>

This notebook resumes the most important things learned in the WorldQuant Applied Data Science Lab course 4 - "Earthquake Damage in Nepal". It uses the courses' project "Earthquake Damage in Kavrepalanchok" since it implements the main ideas learned.

In this project, I'll work with data from Open Data Nepal to build a model to predict building damage from the Nepal 2015 Earthquake. I'll work primarily with data from the Gorkha district, with additional examples from Ramechhap. 

Topics learned in this project:

- How to get data by querying a SQL database.
- How to build a logistic regression model for classification.
- How to build a decision tree model for classification.
- How to incorporate ethical considerations into your model building.

Imports necessary libraries:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted

## Prepare Data

<strong>No-SQL (MongoDB)</strong>:
- database → collection → documents
- semi-structured data (dictionaries are mutable and can have different keys between them)

<strong>SQL database</strong>:
- tables organized in rows
- structured data (each row needs an entry for each column)

### Connect

Connects to the `nepal.sqlite` database using `ipython-sql`:

In [2]:
# loads the extension 'ipythonsql' to connect to the sql database
%load_ext sql

# tells ipythonsql the location of the database
%sql sqlite:///data/nepal.sqlite

We can also connect to the same database using `sqlite3` package:

In [3]:
import sqlite3

conn = sqlite3.connect("data/nepal.sqlite")

There are six common clauses used for querying data:

| Clause Name | Definition | 
| --- | --- | 
| `SELECT` | Determines which columns to include in the query's result |
| `FROM` | Identifies the table from which to query the data from |
| `WHERE` | filters data |
| `GROUP BY` | groups rows by common values in columns |
| `HAVING` | filters out unwanted groups from GROUP BY |
| `ORDER BY` | Orders the rows using one or more columns |
| `LIMIT` | Outputs the specified number of rows |

All clauses may be used together, but `SELECT` and `FROM` are the only required clauses.

Allows us to check everything stored in the database:

In [4]:
%%sql
SELECT *
FROM sqlite_schema

 * sqlite:///data/nepal.sqlite
Done.


[('table', 'id_map', 'id_map', 2, 'CREATE TABLE "id_map" (\n"household_id" INTEGER,\n  "building_id" INTEGER,\n  "vdcmun_id" INTEGER,\n  "district_id" INTEGER\n)'),
 ('index', 'ix_id_map_household_id', 'id_map', 3, 'CREATE INDEX "ix_id_map_household_id"ON "id_map" ("household_id")'),
 ('table', 'building_structure', 'building_structure', 2032, 'CREATE TABLE "building_structure" (\n"building_id" INTEGER,\n  "count_floors_pre_eq" INTEGER,\n  "count_floors_post_eq" INTEGER,\n  "age_building" IN ... (198 characters truncated) ... or_type" TEXT,\n  "other_floor_type" TEXT,\n  "position" TEXT,\n  "plan_configuration" TEXT,\n  "condition_post_eq" TEXT,\n  "superstructure" TEXT\n)'),
 ('index', 'ix_building_structure_building_id', 'building_structure', 2033, 'CREATE INDEX "ix_building_structure_building_id"ON "building_structure" ("building_id")'),
 ('table', 'building_damage', 'building_damage', 12302, 'CREATE TABLE "building_damage" (\n"building_id" INTEGER,\n  "damage_overall_collapse" TEXT,\n  "damage_overall_leaning" TEXT,\n  "damage_overall_adja ... (2923 characters truncated) ... ndslide" INTEGER,\n  "has_geotechnical_risk_rock_fall" INTEGER,\n  "has_geotechnical_risk_flood" INTEGER,\n  "has_geotechnical_risk_other" INTEGER\n)'),
 ('index', 'ix_building_damage_building_id', 'building_damage', 12305, 'CREATE INDEX "ix_building_damage_building_id"ON "building_damage" ("building_id")'),
 ('table', 'household_demographics', 'household_demographics', 30763, 'CREATE TABLE "household_demographics" (\n"household_id" INTEGER,\n  "gender_household_head" TEXT,\n  "age_household_head" REAL,\n  "caste_household"  ... (8 characters truncated) ...  "education_level_household_head" TEXT,\n  "income_level_household" TEXT,\n  "size_household" REAL,\n  "is_bank_account_present_in_household" REAL\n)'),
 ('index', 'ix_household_demographics_household_id', 'household_demographics', 30764, 'CREATE INDEX "ix_household_demographics_household_id"ON "household_demographics" ("household_id")')]

Selects rows from the column `name` in which the `type` column as the value `table`:

In [5]:
# prints the names of the tables in the database
%%sql
SELECT name
FROM sqlite_schema
WHERE type = "table"

SyntaxError: invalid syntax (2798651751.py, line 3)

Determines the unique values in the `district_id` column from table `id_map`:

In [ ]:
%%sql
SELECT distinct(district_id)
FROM id_map

Calculates the number of observations in the `id_map` table associated with district `1`:

In [ ]:
%%sql
SELECT count(*)
FROM id_map
WHERE district_id = 1

Calculates the number of observations in the `id_map` table associated with district `3`:

In [ ]:
%%sql
SELECT count(*)
FROM id_map
WHERE district_id = 3

Joins the unique building IDs from Kavrepalanchok in `id_map`, all the columns from `building_structure`, and the `damage_grade` column from `building_damage`. 

Renames the `building_id` column in `id_map` to `b_id` and limits the results to the first five rows of the new table.

In [ ]:
%%sql

SELECT distinct(i.building_id) as b_id,
  s.*,
  d.damage_grade
FROM id_map as i
JOIN building_structure as s ON i.building_id = s.building_id
JOIN building_damage as d ON i.building_id = d.building_id
WHERE district_id = 3
LIMIT 5

### Import

Builds a wrangle function that uses the query created to create a DataFrame. 

In addition, the function also:

- Creates a `severe_damage` column, where all buildings with a damage grade greater than 3 should be encoded as 1. All other buildings should be encoded at 0.
- Drops columns that could cause issues with leakage or multicollinearity in the model.

In [ ]:
def wrangle(db_path):
    # Connects to database
    conn = sqlite3.connect(db_path)

    # Constructs query
    query = '''
    SELECT distinct(i.building_id) as b_id,
      s.*,
      d.damage_grade
    FROM id_map as i
    JOIN building_structure as s ON i.building_id = s.building_id
    JOIN building_damage as d ON i.building_id = d.building_id
    WHERE district_id = 3
    '''
    
    # Reads query results into DataFrame
    df = pd.read_sql(query, conn, index_col="b_id")

    # Identifies leaky columns (data gathered after earthquake)
    drop_cols = [col for col in df.columns if "post_eq" in col]

    # Adds high-cardinality / redundant column
    drop_cols.append("building_id")

    # Creates binary target column
    df["damage_grade"] = df["damage_grade"].str[-1].astype(int)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)

    # Drops old target
    drop_cols.append("damage_grade")

    # Drops multicollinearity column
    drop_cols.append("count_floors_pre_eq")
    
    # Drops columns
    df.drop(columns=drop_cols, inplace=True)

    return df

In [ ]:
df = wrangle("data/nepal.sqlite")
df.head()

### Explore

Creates a bar chart with the normalized value counts from the `severe_damage` column:

In [ ]:
fig, ax = plt.subplots() 

# Calculate value counts and plot on the axes object
df["severe_damage"].value_counts(normalize=True).plot(
    kind="bar",
    ax=ax  # Direct the plot to our Axes object
)

# Set labels and title 
ax.set_xlabel("Severe Damage")
ax.set_ylabel("Relative Frequency")
ax.set_title("Kavrepalanchok, Class Balance");

Uses seaborn to create a boxplot that shows the distributions of the `plinth_area_sq_ft` column for both groups in the `severe_damage` column:

In [ ]:
fig, ax = plt.subplots() 

# Create the Seaborn boxplot 
sns.boxplot(x="severe_damage", y="plinth_area_sq_ft", data=df, ax=ax)

# Set labels and title
ax.set_xlabel("Severe Damage")
ax.set_ylabel("Plinth Area [sq. ft.]")
ax.set_title("Kavrepalanchok, Plinth Area vs Building Damage");

Creates a pivot table of `df` where the index is `roof_type` and the values come from the `severe_damage` column, aggregated by the mean:

In [ ]:
roof_pivot = pd.pivot_table(
    df, index="roof_type", values="severe_damage", aggfunc=np.mean
).sort_values(by="severe_damage")
roof_pivot

### Split

Creates the feature matrix `X` and target vector `y`:

In [ ]:
X = df.drop(columns="severe_damage")
y = df["severe_damage"]
print("X shape:", X.shape)
print("y shape:", y.shape)

<font size="+1"><strong>Validation Data Split</strong></font>

Validation datasets are usually used to tune model hyperparameters.

A hyperparameter is a model setting that can't be learned during model training and must be explicitly set. In contrast, a model parameter can be learned. An example of a hyperparameter is the depth of a decision tree. 

An example of a model parameter includes a coefficient of a variable from linear regression.


Divides the dataset into training and validation sets using a randomized split:

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

## Build Model

### Baseline

Here the baseline model is represented as if we predicted `severe_damage == 1` for every test datapoint:

In [ ]:
acc_baseline = y_train.value_counts(normalize=True)
print("Baseline Accuracy:", round(acc_baseline, 2))

### Logistic Regression

The logistic regression model is the classifier version of linear regression. It will predict probability values that can be used to assign class labels. 

The model works by taking the output of a linear regression model and feeding it into a sigmoid or logistic function. 

The sigmoid function bounds predictions between 0 and 1, which we then treat as a probability. This allows us to use the model for classification problems.

<code><strong>OneHotEncoder</strong></code>

A property's district is categorical data. 

For many machine learning algorithms, it's common to create a column in a DataFrame to indicate if the feature is present or absent, instead of using the category's name. 

It is necessary so the model can be able to separate the contribution of each category and calculate its coefficients:

### Iterate

Creates the model `model_lr` that uses logistic regression to predict building damage:

In [ ]:
model_lr = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    LogisticRegression(max_iter=1000)
).fit(X_train, y_train)

Calculates training and validation accuracy score for `model_lr`:

In [ ]:
lr_train_acc = accuracy_score(y_train, model_lr.predict(X_train))
lr_val_acc = model_lr.score(X_val, y_val)

print("Logistic Regression, Training Accuracy Score:", lr_train_acc)
print("Logistic Regression, Validation Accuracy Score:", lr_val_acc)

### Decision Trees

Decision trees are a general class of machine learning models that are used for both classification and regression. 

The model resemble a tree, complete with branches and leaves. The model is essentially a series of questions with "yes" or "no" answers. 

The decision tree starts by checking whatever condition does the best job at correctly separating the data into the two classes in the binary target. It then progressively checks more conditions until it can predict an observation's label. 

They are popular because they are more flexible than linear models and intuitive in a way that makes them easy to explain to stakeholders who are not familiar with data science.

We start at the root in a place of dataset impurity and our goal is to divide this set into categories constantly increasing pureness in the data.

Decision trees pros and cons:

| Pros | Cons | 
| --- | --- | 
| can be used for classification and regression | generalization: they are prone to overfitting |
| handles both numerical and categorical data | robustness: small variations in data can result in a different tree |
| models nonlinear relationships between the features and target | class imbalance: if one class is much larger than the other, the tree may be unbalanced |

<code><strong>Ordinal Encoder</strong></code>

Two primary issues that can arise using `OneHotEncoder` as the number of features grows: computational complexity (operations performed on larger datasets may take longer) and overfitting (the model may not generalize to new data).

Ordinal encoding is a popular choice for encoding the categorical variable. Instead of creating new columns, ordinal encoding simply replaces the categories in a categorical variable with integers.

### Iterate

Creates a `for` loop to train and evaluate the model `model_dt` at all depths from 1 to 15:

In [ ]:
depth_hyperparams = range(1, 16)
training_acc = []
validation_acc = []
for d in depth_hyperparams:
    model_dt = make_pipeline(
        OrdinalEncoder(),
        DecisionTreeClassifier(max_depth=d, random_state=42)
    )
    model_dt.fit(X_train, y_train)
    training_acc.append(model_dt.score(X_train, y_train))
    validation_acc.append(model_dt.score(X_val, y_val))

In [ ]:
pd.Series(validation_acc, index=depth_hyperparams)

Plots the validation curve for `model_dt`:

In [ ]:
fig, ax = plt.subplots() 

#  Plot the training accuracy on the axes object
ax.plot(depth_hyperparams, training_acc, label="training")

#  Plot the validation accuracy on the same axes object
ax.plot(depth_hyperparams, validation_acc, label="validation") 

#  Set labels and title  
ax.set_xlabel("Max Depth")
ax.set_ylabel("Accuracy Score")
ax.set_title("Validation Curve, Decision Tree Model")

# Add the legend 
ax.legend()

Builds and trains a new decision tree model `final_model_dt`, using the value for `max_depth` that yielded the best validation accuracy score in the above plot:

In [ ]:
final_model_dt = make_pipeline(
    OrdinalEncoder(),
    DecisionTreeClassifier(max_depth=10, random_state=42)
).fit(X, y)

### Evaluate

Reads the CSV file with the test set into a DataFrame. 

Next, uses `final_model_dt` to generate a list of test predictions `y_test_pred`:

In [ ]:
X_test = pd.read_csv("data/kavrepalanchok-test-features.csv", index_col="b_id")
y_test_pred = final_model_dt.predict(X_test[X_train.columns])
y_test_pred

## Communicate Results

Creates a Series Gini `feat_imp` sorted from smallest to largest feature importance:

In [ ]:
features = X_train.columns
importances = final_model_dt.named_steps["decisiontreeclassifier"].feature_importances_

feat_imp = pd.Series(importances, index=features).sort_values()
feat_imp.head()

Creates a horizontal bar chart of `feat_imp`:

In [ ]:
fig, ax = plt.subplots() 

# Create the horizontal bar plot on the axes object
feat_imp.plot(kind="barh", ax=ax)

# Set labels and title 
ax.set_xlabel("Gini Importance")
ax.set_ylabel("Feature")
ax.set_title("Kavrepalanchok Decision Tree, Feature Importance")

# Apply tight layout 
fig.tight_layout()